In [3]:
import pandas as pd
import json

In [12]:
flatten = lambda x: [j for i in x if i for j in i]

## CAS番号とwikipediaID（Page_id）の対応テーブル作成

In [3]:
with open("../data/cas_number.json", 'r') as f:
    extraction_cas_num = json.load(f)

In [15]:
print(len(flatten(list(extraction_cas_num.values()))))
print(len(set(flatten(list(extraction_cas_num.values())))))

5342
5281


- CAS番号が重複している
- おそらく，記事内に異性体のCAS番号が載せられているかも -> esaに調査結果まとめた

In [22]:
# CAS to page_id
cas2pageId = {}
for page_id, cas_list in extraction_cas_num.items():
    for cas in cas_list:
        if cas2pageId.get(cas):
            cas2pageId[cas].append(page_id)
        else:
            cas2pageId[cas] = [page_id]

with open("../data/cas2pageID.json", 'w') as f:
    json.dump(cas2pageId, f)

## wikipediaID（Page_id）とWikidataIDの対応テーブル作成

In [23]:
with open("../data/pageID2wikidataID.json", 'r') as f:
    p2w = json.load(f)

In [25]:
with open("../data/wikidataID2pageID.json", 'w') as f:
    json.dump({v: k for k, v in p2w.items()}, f)

## CAS番号とCIDの対応テーブル作成

In [30]:
def get_CID(articles: dict):
    return [str(article.get('Record').get('RecordNumber')) for article in list(articles.values())[0]]

In [38]:
# CAS number to CID
cas2cid = {}
with open("../data/pubchem_articles.jsonl", 'r') as f:
    count = 0
    line = f.readline()
    while line:
        articles = json.loads(line)
        cas = list(articles.keys())[0]
        cas2cid[cas] = get_CID(articles)
        
        line = f.readline()

In [41]:
print(len(flatten(cas2cid.values())))
print(len(set(flatten(cas2cid.values()))))

6726
6624


- CAS番号からCIDは一意に決まらない

In [43]:
cid2cas = {}
for cas, cid_list in cas2cid.items():
    for cid in cid_list:
        if cid2cas.get(cid):
            cid2cas[cid].append(cas)
        else:
            cid2cas[cid] = [cas]

In [45]:
with open("../data/cas2cid.json", 'w') as f:
    json.dump(cas2cid, f)
    
with open("../data/cid2cas.json", 'w') as f:
    json.dump(cid2cas, f)